Load the database

In [10]:
from sqlalchemy import create_engine
import pandas as pd

# Connection parameters
dbname = 'students'
user = 'students_codealong'
password = 'students_codealong'
host = 'workspacedemodb.datacamp.com'
port = '5432'

# Create the connection string
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Execute the query and load the result into a pandas DataFrame
df = pd.read_sql("SELECT * FROM public.students", engine)

Perform Queries

In [17]:
query = """
SELECT english_cate, age
FROM public.students;
"""
selected_columns_df = pd.read_sql(query, engine)
print(selected_columns_df)

    english_cate   age
0           High  24.0
1           High  28.0
2           High  25.0
3        Average  29.0
4        Average  28.0
..           ...   ...
281                NaN
282                NaN
283                NaN
284                NaN
285                NaN

[286 rows x 2 columns]


How many international and domestic students are in the data set?

In [18]:
query = """
SELECT inter_dom, COUNT(*) AS count_inter_dom
FROM public.students
GROUP BY inter_dom;
"""
grouped_df = pd.read_sql(query, engine)
print(grouped_df.head())


  inter_dom  count_inter_dom
0       Dom               67
1                         18
2     Inter              201


Look into the 18 unassigned rows to understand what they could be.

In [19]:
query = """
SELECT *
FROM public.students
WHERE inter_dom NOT LIKE 'Dom' AND inter_dom NOT LIKE 'Inter';
"""
filtered_df = pd.read_sql(query, engine)
print(filtered_df)

   inter_dom region gender academic   age age_cate  stay stay_cate japanese  \
0                                    None     None  None               None   
1                                    None     None  None               None   
2                                    None     None  None               None   
3                                    None     None  None               None   
4                                    None     None  None               None   
5                                    None     None  None               None   
6                                    None     None  None               None   
7                                    None     None  None               None   
8                                    None     None  None               None   
9                                    None     None  None               None   
10                                   None     None  None               None   
11                                   None     None  

Where are the international students from?

In [16]:
# Define your SQL query
query = """
SELECT region, COUNT(inter_dom) as count_inter_dom
FROM public.students
WHERE inter_dom = 'Inter'
GROUP BY region;
"""
result_df = pd.read_sql(query, engine)
print(result_df)

   region  count_inter_dom
0      SA               18
1      EA               48
2     JAP                2
3  Others               11
4     SEA              122


Find the minimum, maximum, and average of each of the diagnostic tests (PHQ-9, SCS, ASISS). This information is in the paper, but it's good practice to look this up yourself during analysis.


In [20]:
# Define your SQL query
query = """
SELECT MIN(ToDep) AS min_phq,
	MAX(ToDep) AS max_phq,
	ROUND(AVG(ToDep), 2) AS avg_phq,
	MIN(ToSC) AS min_scs,
	MAX(ToSC) AS max_scs,
	ROUND(AVG(ToSC), 2) AS avg_scs,
	MIN(ToAS) AS min_as,
	MAX(ToAS) AS max_as,
	ROUND(AVG(ToAS), 2) AS avg_as
FROM students;
"""
result_df = pd.read_sql(query, engine)
print(result_df)

   min_phq  max_phq  avg_phq  min_scs  max_scs  avg_scs  min_as  max_as  \
0        0       25     8.19        8       48    37.47      36     145   

   avg_as  
0   72.38  


What are the summary statistics for domestic students and international students?

In [21]:
# Define your SQL query
query = """
SELECT inter_dom,
	MIN(ToDep) AS min_phq,
	MAX(ToDep) AS max_phq,
	ROUND(AVG(ToDep), 2) AS avg_phq,
	MIN(ToSC) AS min_scs,
	MAX(ToSC) AS max_scs,
	ROUND(AVG(ToSC), 2) AS avg_scs,
	MIN(ToAS) AS min_as,
	MAX(ToAS) AS max_as,
	ROUND(AVG(ToAS), 2) AS avg_as
FROM students
WHERE inter_dom IN ('Inter', 'Dom')
GROUP BY inter_dom;
"""
result_df = pd.read_sql(query, engine)
print(result_df)

  inter_dom  min_phq  max_phq  avg_phq  min_scs  max_scs  avg_scs  min_as  \
0     Inter        0       25     8.04       11       48    37.42      36   
1       Dom        0       23     8.61        8       48    37.64      36   

   max_as  avg_as  
0     145   75.56  
1     112   62.84  


How does the age of the international student impact the scores?

In [22]:
# Define your SQL query
query = """
SELECT age,
ROUND(AVG(todep), 2) AS avg_phq,
ROUND(AVG(tosc), 2) AS avg_scs,
ROUND(AVG(toas), 2) AS avg_as
FROM public.students
WHERE inter_dom = 'Inter'
GROUP BY age
ORDER BY age;
"""
result_df = pd.read_sql(query, engine)
print(result_df)

    age  avg_phq  avg_scs  avg_as
0    17     4.67    37.33   70.67
1    18     8.75    34.11   80.61
2    19     8.44    37.90   74.10
3    20     7.35    38.21   73.26
4    21     9.23    37.74   75.23
5    22     8.36    38.14   70.43
6    23     9.67    32.00   81.25
7    24     4.67    42.33   74.33
8    25     6.11    37.33   80.78
9    27    10.00    35.00   42.00
10   28     3.33    39.00   71.00
11   29     3.75    43.00   63.50
12   30     9.33    41.00   97.33
13   31     5.75    43.50   80.25


See how another variable may impact the score.

In [23]:
# Define your SQL query
query = """
SELECT stay,
ROUND(AVG(todep), 2) AS avg_phq,
ROUND(AVG(tosc), 2) AS avg_scs,
ROUND(AVG(toas), 2) AS avg_as
FROM public.students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay;
"""
result_df = pd.read_sql(query, engine)
print(result_df)

   stay  avg_phq  avg_scs  avg_as
0     1     7.48    38.11   72.80
1     2     8.28    37.08   77.67
2     3     9.09    37.13   78.00
3     4     8.57    33.93   87.71
4     5     0.00    34.00   91.00
5     6     6.00    38.00   58.67
6     7     4.00    48.00   45.00
7     8    10.00    44.00   65.00
8    10    13.00    32.00   50.00


In [25]:
# Import plotly packages
import plotly.express as px
import plotly.graph_objects as go

# Create the histogram figure
fig = px.histogram(
    
# Select the dataframe
    data_frame=df,
    
# Select the column you want to visualize
    x="todep",

# Select the additional column to determine different groups of data
    color="inter_dom",

# Clean up the labels
    title="Distribution of the total scores from the PHQ-9 test",
    labels={"inter_dom":"Type of Students", "todep":"Total PHQ-9 Scores"})

# Show the figure
fig.show()

In [26]:
# Create the box plot figure
fig = px.box(
    
# Select the dataframe
    data_frame=df,

# Select the column you want to visualize
    x="todep",
    
# Select the additional column to determine different groups of data    
    color="inter_dom",

# Select the additional column to determine what information is shown when you hover over the plot
    hover_data=["age"],

# Select the option to view all data points
    points="all",

# Clean up the labels
    title="Box plot of the total scores from the PHQ-9 test",
    labels={"inter_dom":"Type of Student", "ToDep":"Total PHQ-9 Scores"})

# Show the figure
fig.show()

In [30]:
import pandas as pd
import plotly.graph_objects as go

# List of continuous variables
continuous_variables = ['age', 'stay', 'japanese', 'english', 'todep', 'tosc', 'apd', 'ahome', 'aph', 'afear', 'acs', 'aguilt', 'amiscell', 'toas', 'partner', 'friends', 'parents', 'relative', 'profess', 'phone', 'doctor', 'reli', 'alone', 'others', 'internet']

# Check which columns are actually present in the DataFrame
available_continuous_variables = [col for col in continuous_variables if col in df.columns]

# Create a subset DataFrame with only the columns of continuous variables that are present
data_cont = df[available_continuous_variables]

# Convert columns to numeric, coercing errors to NaN
data_cont = data_cont.apply(pd.to_numeric, errors='coerce')

# Drop rows with any NaN values (optional, depending on how you want to handle missing data)
data_cont = data_cont.dropna()

# Create a Pearson correlation matrix
data_corr = data_cont.corr(method='pearson')

# Build the heatmap
fig = go.Figure(go.Heatmap(
    x=data_corr.columns, 
    y=data_corr.columns, 
    z=data_corr.values,
    zmin=-1, 
    zmax=1
))

# Adjust the plot size
fig.update_layout(width=900, height=900)

# Show the plot
fig.show()
